In [3]:
import pandas as pd
pd.set_option('display.max_columns', None)
import pyreadstat as st
import numpy as np
import matplotlib.pyplot as plt


path = r"C:\Users\HP\OneDrive\Escritorio\David Guzzi\DiTella\MEC\Materias\2025\2025 1T\[MT09] Econometría de Datos de Panel\Clases prácticas\PS 0-20250420\ps0_data\sim_1.dta"

df, meta = st.read_dta(path)
df

,x2,x3,e,u,y
0,6.954513,4.884080,-5.819671,-7.075903,5.762690
1,8.537341,-4.902815,3.898010,5.552234,10.186760
2,1.739996,9.951114,-5.602970,-4.044251,8.646858
3,-3.906954,-3.913731,5.773367,2.369219,-4.451466
4,2.368236,-2.692694,4.975786,3.824420,4.499962
5,-3.726072,-0.859068,-7.732616,-3.231157,-6.816297
6,3.618390,3.942587,3.645388,3.174982,11.735958
7,9.037177,2.183038,-2.760063,-4.132867,8.087349
8,4.197956,1.401212,2.532652,2.337455,8.936624
9,2.557375,-2.210001,-2.918557,-2.286053,-0.938679


In [4]:
import statsmodels.api as sm

y = df['y']
x = df[['x2', 'x3']]
X = sm.add_constant(x)

model = sm.OLS(y,X)
results = model.fit()

results.summary()

c:\Users\HP\AppData\Local\Programs\Python\Python312\Lib\site-packages\scipy\stats\_axis_nan_policy.py:418: UserWarning: `kurtosistest` p-value may be inaccurate with fewer than 20 observations; only n=10 observations were given.
  return hypotest_fun_in(*args, **kwds)


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.713
Model:                            OLS   Adj. R-squared:                  0.631
Method:                 Least Squares   F-statistic:                     8.698
Date:                Wed, 07 May 2025   Prob (F-statistic):             0.0127
Time:                        21:09:30   Log-Likelihood:                -26.052
No. Observations:                  10   AIC:                             58.10
Df Residuals:                       7   BIC:                             59.01
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.9142      1.542      0.593      0.572      -2.731       4.560
x2             1.0528      0.296      3.552      0.009       0.352       1.754
x3             0.4464      0.287      1.555      0.164      -0.233       1.125
==============================================================================
Omnibus:                        2.262   Durbin-Watson:                   2.981
Prob(Omnibus):                  0.323   Jarque-Bera (JB):                0.867
Skew:                          -0.059   Prob(JB):                        0.648
Kurtosis:                       1.562   Cond. No.                         6.98
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [6]:
# A) Test univariante: totchr = 0 (equivale a `test totchr=0`)
t_test_totchr = results.t_test('x3 = 1')
print("\nTest x3 = 1:")
print(t_test_totchr)


Test x3 = 1:
                             Test for Constraints                             
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
c0             0.4464      0.287     -1.928      0.095      -0.233       1.125


In [ ]:
# Test F equivalente (aunque sea una sola restricción)
f_test_result = results.f_test("x3 = 1")
print(f_test_result)

<F test: F=3.717906856224962, p=0.0951759293690687, df_denom=7, df_num=1>


In [19]:
import numpy as np
import statsmodels.api as sm

y = df['y']
x = df[['x2', 'x3']]
X = sm.add_constant(x)

# Generar el vector de pesos
w = np.exp(-1 + 0.2 * df['x2'])



# Ajustar el modelo WLS
model = sm.WLS(y, X, weights=1/w)
results = model.fit()

# Mostrar resumen del modelo
print(results.summary())

                            WLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.838
Model:                            WLS   Adj. R-squared:                  0.791
Method:                 Least Squares   F-statistic:                     18.08
Date:                Wed, 07 May 2025   Prob (F-statistic):            0.00172
Time:                        21:25:49   Log-Likelihood:                -25.951
No. Observations:                  10   AIC:                             57.90
Df Residuals:                       7   BIC:                             58.81
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.6061      1.027      0.590      0.5

c:\Users\HP\AppData\Local\Programs\Python\Python312\Lib\site-packages\scipy\stats\_axis_nan_policy.py:418: UserWarning: `kurtosistest` p-value may be inaccurate with fewer than 20 observations; only n=10 observations were given.
  return hypotest_fun_in(*args, **kwds)


In [20]:
# Realizar el test de hipótesis: H0: coeficiente de x3 = 1
f_test_result = results.f_test("x3 = 1")
print(f_test_result)


<F test: F=3.656912567307812, p=0.09742655997334931, df_denom=7, df_num=1>
